In [1]:
import pandas as pd
import numpy as np
import warnings
from Dataset_functions import *
# Suppress all warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [2]:
premier_league_season24_25 = pd.read_csv('24_25_eng.csv')
premier_league_season23_24 = pd.read_csv('23_24_eng.csv')
premier_league_season22_23 = pd.read_csv('22_23_eng.csv')
premier_league_season21_22 = pd.read_csv('21_22_eng.csv')
premier_league_season20_21 = pd.read_csv('20_21_eng.csv')
premier_league_season19_20 = pd.read_csv('19_20_eng.csv')
premier_league_season18_19 = pd.read_csv('18_19_eng.csv')
premier_league_season17_18 = pd.read_csv('17_18_eng.csv')


In [3]:
def prepare_dataframe(csv_file, season):
    season_data = pd.read_csv(csv_file)
    teams = season_data['HomeTeam'].unique()
    url = f"https://fbref.com/en/comps/9/{season}/schedule/{season}-Premier-League-Scores-and-Fixtures"
    season_data = join_scraped(url, season_data, season, teams)
    season_data = clean_v2(season_data)
    season_data = create_seasonal_table(season_data,teams)
    return season_data

In [5]:
season = '2024-2025'

In [7]:
premier_league_season24_25 = prepare_dataframe('24_25_eng.csv', '2024-2025')

In [8]:
premier_league_season24_25

,Div,Date,Time,FTHG,FTAG,B365H,B365D,B365A,Home xG,Away xG,HomeTeam_Arsenal,HomeTeam_Aston Villa,HomeTeam_Bournemouth,HomeTeam_Brentford,HomeTeam_Brighton,HomeTeam_Chelsea,HomeTeam_Crystal Palace,HomeTeam_Everton,HomeTeam_Fulham,HomeTeam_Ipswich,HomeTeam_Leicester,HomeTeam_Liverpool,HomeTeam_Man City,HomeTeam_Man United,HomeTeam_Newcastle,HomeTeam_Nott'm Forest,HomeTeam_Southampton,HomeTeam_Tottenham,HomeTeam_West Ham,HomeTeam_Wolves,AwayTeam_Arsenal,AwayTeam_Aston Villa,AwayTeam_Bournemouth,AwayTeam_Brentford,AwayTeam_Brighton,AwayTeam_Chelsea,AwayTeam_Crystal Palace,AwayTeam_Everton,AwayTeam_Fulham,AwayTeam_Ipswich,AwayTeam_Leicester,AwayTeam_Liverpool,AwayTeam_Man City,AwayTeam_Man United,AwayTeam_Newcastle,AwayTeam_Nott'm Forest,AwayTeam_Southampton,AwayTeam_Tottenham,AwayTeam_West Ham,AwayTeam_Wolves,matchday,total_points,total_points_home_team,total_points_away_team,5_form,5_form_home_team,5_form_away_team,10_form,10_form_home_team,10_form_away_team,total_goals,total_goals_home_team,total_goals_away_team,total_goals_against,total_goals_against_home_team,total_goals_against_away_team,total_goals_per_game,total_goals_per_game_home_team,total_goals_per_game_away_team,total_goals_against_per_game,total_goals_against_per_game_home_team,total_goals_against_per_game_away_team,5_form_goals_scored,5_form_goals_scored_home_team,5_form_goals_scored_away_team,5_form_goals_against,5_form_goals_against_home_team,5_form_goals_against_away_team,10_form_goals_scored,10_form_goals_scored_home_team,10_form_goals_scored_away_team,10_form_goals_against,10_form_goals_against_home_team,10_form_goals_against_away_team,total_shots_per_game,total_shots_per_game_home_team,total_shots_per_game_away_team,total_shots_against_per_game,total_shots_against_per_game_home_team,total_shots_against_per_game_away_team,total_shots_on_target_per_game,total_shots_on_target_per_game_home_team,total_shots_on_target_per_game_away_team,total_shots_on_target_against_per_game,total_shots_on_target_against_per_game_home_team,total_shots_on_target_against_per_game_away_team,5_form_shots,5_form_shots_home_team,5_form_shots_away_team,5_form_shots_against,5_form_shots_against_home_team,5_form_shots_against_away_team,10_form_shots,10_form_shots_home_team,10_form_shots_away_team,10_form_shots_against,10_form_shots_against_home_team,10_form_shots_against_away_team,5_form_shots_on_target,5_form_shots_on_target_home_team,5_form_shots_on_target_away_team,5_form_shots_on_target_against,5_form_shots_on_target_against_home_team,5_form_shots_on_target_against_away_team,10_form_shots_on_target,10_form_shots_on_target_home_team,10_form_shots_on_target_away_team,10_form_shots_on_target_against,10_form_shots_on_target_against_home_team,10_form_shots_on_target_against_away_team,total_yellow_cards_per_game,total_yellow_cards_per_game_home_team,total_yellow_cards_per_game_away_team,total_yellow_cards_against_per_game,total_yellow_cards_against_per_game_home_team,total_yellow_cards_against_per_game_away_team,total_red_cards_per_game,total_red_cards_per_game_home_team,total_red_cards_per_game_away_team,total_red_cards_against_per_game,total_red_cards_against_per_game_home_team,total_red_cards_against_per_game_away_team,total_xg_per_game,total_xg_per_game_home_team,total_xg_per_game_away_team,total_xg_against_per_game,total_xg_against_per_game_home_team,total_xg_against_per_game_away_team,5_form_xg,5_form_xg_home_team,5_form_xg_away_team,5_form_xg_against,5_form_xg_against_home_team,5_form_xg_against_away_team,10_form_xg,10_form_xg_home_team,10_form_xg_away_team,10_form_xg_against,10_form_xg_against_home_team,10_form_xg_against_away_team,total_points_away,total_points_home,5_form_away,10_form_away,5_form_home,10_form_home,total_goals_away,total_goals_against_away,total_goals_per_game_away,total_goals_against_per_game_away,total_goals_home,total_goals_against_home,total_goals_per_game_home,total_goals_against_per_game_home,5_form_goals_scored_away,10_form_goals_scored_away,5_form_goals_agains

# co dodać

Target ---> wyliczenie prawdopodobieństwa każdego z 1x2 (opcjonalnie porówanie strzelonych bramek) i porównanie z kursasmi/prawdziwymi wynikami?

# oddsy na prawdopodobieństwo
# wartość lineupów /suma wages zawodników?
# średnia zdobywanych/traconych bramek w sezonie(poprzednich)
# pozycja w tabeli >>>>>>>> to samo co punkty
# podania total/dom/wyjazd średnia z ost 5/srednia sezonowa 
# posiadanie piłki dom/wyjazd
# forma dom/wyjazd -- GOTOWE
# brasmki drużyny i przeciwko -- GOTOWE
# xg
# strzały ------ ---
# kartki ----------
# h2h drużyny powyższe 
# punkty i forma ostatnich 5 praeciwników --- 